**Содержание**<a id='toc0_'></a>    
- [Множественное наследование](#toc1_)    
  - [Указатели на базовые классы при множ.наследовании](#toc1_1_)    
  - [Создание и удаление объекта при множ.наследовании](#toc1_2_)    
  - [Виртуальное наследование](#toc1_3_)    
    - [Как устроено расположение в памяти объектов виртуального наследования](#toc1_3_1_)    
    - [Кто вызывает конструктор виртуального базового класса?](#toc1_3_2_)    
  - [Выводы](#toc1_4_)    
- [Преобразование типов в стиле C++](#toc2_)    
  - [Преобразование в стиле C](#toc2_1_)    
  - [static_cast](#toc2_2_)    
  - [const_cast](#toc2_3_)    
  - [reinterpret_cast](#toc2_4_)    
  - [Границы применимости преобразования в стиле C](#toc2_5_)    
  - [Когда преобразование в стиле C приводит к ошибке](#toc2_6_)    
- [Информация о типах времени выполнения](#toc3_)    
  - [Run Time Type Information (RTTI)](#toc3_1_)    
  - [Использование typeid и type_info](#toc3_2_)    
  - [dynamic_cast](#toc3_3_)    
  - [Когда можно избежать использования RTTI](#toc3_4_)    
- [Указатели на функции](#toc4_)    
  - [Указатели на функции](#toc4_1_)    
  - [Указатели на функции: параметризация алгоритмов](#toc4_2_)    
  - [О полезности typedef](#toc4_3_)    
  - [Шаблон проектирования Listener](#toc4_4_)    
  - [Пример для самостоятельных экспериментов](#toc4_5_)    
  - [Пример для самостоятельных экспериментов 2](#toc4_6_)    
- [Указатели на методы и поля класса](#toc5_)    
  - [Указатели на методы: параметризация алгоритмов](#toc5_1_)    
  - [Указатели на поля: параметризация алгоритмов](#toc5_2_)    
  - [Резюме по синтаксису](#toc5_3_)    
  - [Как такие указатели устроены?](#toc5_4_)    
  - [Зачем нужно смещение?](#toc5_5_)    
  - [Важные моменты](#toc5_6_)    
    - [Идеома safe bool](#toc5_6_1_)    
  - [Пространства имён](#toc5_7_)    
    - [Пространства имён](#toc5_7_1_)    
  - [Примеры](#toc5_8_)    
  - [Описание пространства имён](#toc5_9_)    
  - [Доступ к именам](#toc5_10_)    
  - [Поиск имён](#toc5_11_)    
  - [Ключевое слово using](#toc5_12_)    
  - [Поиск Кёнига](#toc5_13_)    
  - [Безымянный namespace](#toc5_14_)    
  - [Заключение](#toc5_15_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Множественное наследование](#toc0_)

Множественное наследование (multiple inheritance) — возможность наследовать сразу несколько классов.

Пример:

    struct Unit {
        Unit(unitid id, int hp): id_(id), hp_(hp) {}
        virtual unitid id() const { return id_; }
        virtual int hp() const { return hp_; }
    private:
        unitid id_;
        int hp_;
    };

    struct Elf: Unit { ... };
    struct Archer: Unit { ... };
    struct ElfArcher: Elf, Archer {
        unitid id() const { return Elf::id(); } // фокус: указали из какого предка брать методы 
        int hp() const { return Elf::hp(); }
    };

`Unit` является предком `Elf` и `Archer`, поэтому имеется неопределенность, откуда брать методы, унаследованные от `Unit`
- решение - преопределить в потомке эти вирт.методы и пренаправить их к конкретному предку
- а что будет, если эти данные в предках `ElfArcher` рассинхронизируются (вряд ли это касается `id()`, но `hp()` вполне может!)

## <a id='toc1_1_'></a>[Указатели на базовые классы при множ.наследовании](#toc0_)

Важно: при приведении от производного класса к базовым указатели могут смещаться:

- В памяти объект ElfArcher будет выглядеть так:
```
    |Unit|Elf|Unit|Archer| собственные данные ElfArcher |
    ^        ^
    Elf*     Archer*
    ElfArcher*
```
- Компилятор сможет преобразовать `ElfArcher*` к `Archer*`, умеет посчитать сдвиг указателя.
- Ошибка (в виде UB) возникает если преобразовать `Archer*` в число, а потом к `ElfArcher*`, обратный сдвиг он посчитать не сможет и получится, что `ElfArcher*` будет указывать на ячейку `Archer*`
    - если преобразовывать просто к указателю, то никаких проблем возникать не будет, только если слишком мудрить с указателями, преобразуя их по значению (не очень актуальная проблема в общем)
```
ElfArcher*: 0x961eb0
-> cast to Elf*   : 0x961eb0
-> cast to Archer*: 0x961ec8

Archer HP (UB, должно быть 42): 100

ElfArcher* from Elf*: 0x961eb0
ElfArcher HP: 100
ElfArcher* from Archer*: 0x961eb0
ElfArcher HP: 100

(ElfArcher*)archer: 0x961eb0
(ElfArcher*)(void *)archer: 0x961ec8
ElfArcher HP (ожидалось UB, но его нет): 100
```
- даже некорректное приведение типа указателя не ломает логику, 
- но почему-то корректное приведение к `Archer*` логику нарушает и метод `hp()` все равно берется из `Elf`, как это предусмотрено классом `ElfArcher`, хотя мы явно сказали к какому типу мы хотим привести указатель и корректно привели...
    - непонятно...

## <a id='toc1_2_'></a>[Создание и удаление объекта при множ.наследовании](#toc0_)

Обход дерева классов для вызова конструкторов осуществляется в глубину слева направо.

Допустим структра наследования выглядит так: `E(B(A),C,D(A, C))`

- Порядок вызова конструкторов: `A B C A C D E`
- Деструкторы всегда вызываются в обратном порядке

Проблемы:
1. Дублирование данных классов A и C в классе потомке E
- это может быть проблемой, но это может быть наоборот, желательным поведением
2. Недоступность из E методов первого вхождения класса C (непосредственного предка E)
- если в C определена `foo()`, то вызов `E.foo()` неопределен, т.к. компилятор не знает из какого варианта C брать этот метод
- мы можем в привести указатель на E к типу указателя на D и получить доступ к `foo()` из С, являющегося предком D (если очень хочется, то можно затем явно привести к типу C, но это уже не обязательно)
```
E e;
D & d = e;
d.foo(); 
// C & c = d;
// c.foo();
```
- но нет никаких явных способов привести указатель на E к типу указателя на C, так чтобы он ссылался на непосредственного предка E. Если в этом предке хранятся какие-то нужные данные, мы к ним не можем получить доступ из E

Код с приведением типа - просто варнинг: `warning: direct base ‘C’ inaccessible in ‘E’ due to ambiguity`

Попытка вызвать `e.foo()`: `error: request for member ‘foo’ is ambiguous`



**Задача**

Вам дана следующая иерархия классов:

    struct Base { int i; }; 
    struct D1 : Base { ... };
    struct D2 : Base { ... }; 
    struct D3 : D1, D2 { ... };

Напишите функцию D1BaseToD2Base, которая преобразует указатель типа Base на объект типа D3, который ссылается на экземпляр Base, соответствующий D1, в указатель, ссылающийся на экземпляр Base соответствующий D2.

    Base const * D1BaseToD2Base( Base const * base )
    {
        return (Base const *)(D2 *)(D3 *)(D1 *)base;

        // как вариант: будет неявно преобразован к Base const * 
        // return (D2*)(D3*)(D1*) base;

        // как вариант: без C-style cast
        // return static_cast<Base const*>(static_cast<D2 const*>(static_cast<D3 const*>(static_cast<D1 const*>(base))));

        // по смыслу классы должны быть полиморфными
        // поэтому самый корректный вариант будет 
        // return dynamic_cast<Base const *> (dynamic_cast<D2 const *> (dynamic_cast<D3 const *> (base)));
    }

    int main() {
        D3 * d3 = new D3();
        D1 * d1 = d3;
        Base * b1 = d1;

        std::cout << "D3:D1:Base:" << b1 << std::endl;
        std::cout << "D3:D2:Base:" << D1BaseToD2Base(b1) << std::endl;


        return 0;
    }


Вывод:

    D3:D1:Base:0x558d247b5eb0
    D3:D2:Base:0x558d247b5eb4

## <a id='toc1_3_'></a>[Виртуальное наследование](#toc0_)

То что данные выше дублируются (`Unit` и др. при наследовании от разных родителей) - это явный архитектурный баг (неэффективное использование памяти, возможность рассинхронизации данных в разных классах).

Но это может быть и специльно задуманной фичей (например, разные классы автора):

`Song(Lyrics(Author), Music(Author))`

Если мы все же хотим, как в первом случае, добиться, чтобы исходный базовый класс участвовал в наследовании только 1 раз, то используется **виртуальное наследование**

- чтобы класс унаследовался виртуально **все** его вхождения в дерево наследования, которые должны быть общими, должны быть с ключевым словом `virtual`

```
struct Unit {};
struct Elf: virtual Unit {};
struct Archer: virtual Unit {};
struct ElfArcher: Elf, Archer {};
```
- в результате получается ромбовидная структура наследование (diamond inheritance)
- если нужно, допускается, чтобы в определенных предках базовый класс был общими, а в некоторых - собственные (не виртуальные)

**Задача**

Вам дана следующая иерархия классов:

    struct Base {};
    struct D1 : Base {}; // 1
    struct D2 : Base {}; // 2
    struct D3 : Base {}; // 3
    struct D4 : Base {}; // 4
    struct D5 : D1, D2, D3, D4 {}; // 5

В каких строках нужно добавить ключевое слово virtual, если мы хотим, чтобы у D1, D2 и D4 был общий экземпляр Base, а у D3 — собственный.

    D1, D2 и D4

А есть ли хочется, например, для D1 и D2 сделать свой общий Base, а для D3 и D4 другой, но тоже общий для них двух?
- облом, делать фиктивные классы D12 : D1, D2 и D34 : D3, D4
- либо, делать разные Base1, Base2

### <a id='toc1_3_1_'></a>[Как устроено расположение в памяти объектов виртуального наследования](#toc0_)

Имеем?:

    Elf: |Unit|Elf|
    Archer: |Unit|Archer|

Так?
    
    ElfArcher: |Unit|Elf|Archer|ElfArcher|

- неа, так станет невозможным привести к типу указателя на Archer

Так?
    
    ElfArcher: |Elf|Unit|Archer|ElfArcher|

- неа, так станет невозможным привести к типу указателя на Elf

Вообще не так! Даже базовые классы располагаются иначе!

При **виртуальном наследовании** имеем:

    Elf: |Elf|Unit|
    Archer: |Archer|Unit|  
    ElfArcher: |Elf|Archer|Unit|ElfArcher|

- тогда виртуальная база располагается не в начале класса, как всегда происходит при обычном наследовании, а в некотором другом месте
- не обязательно в конце базового класса, но некоторые компиляторы именно так и делают

Укладка классов в памяти: 

`g++ -fdump-lang-class -c multinher_ptr.cpp`

Тут какие-то свои нюансы с укладкой, все не очевидно

В `clang++` вроде отображается как в лекции:

`clang++ -cc1 -fdump-record-layouts multinher_ptr.cpp`

Обычное наследование

    *** Dumping AST Record Layout
         0 | struct ElfArcher (empty)
         0 |   struct Elf (base) (empty)
         0 |     struct Unit (base) (empty)
         1 |   struct Archer (base) (empty)
         1 |     struct Unit (base) (empty)
           | [sizeof=2, dsize=0, align=1,
           |  nvsize=2, nvalign=1]

Виртуальное:

    *** Dumping AST Record Layout
         0 | struct ElfArcher
         0 |   struct Elf (primary base)
         0 |     (Elf vtable pointer)
         8 |   struct Archer (base)
         8 |     (Archer vtable pointer)
         0 |   struct Unit (virtual base) (empty)
           | [sizeof=16, dsize=16, align=8,
           |  nvsize=16, nvalign=8]



Допустим тип объекта определяется в рантайме:

    Elf * e = (rand() % 2) ? new Elf() : new ElfArcher()
    unitid id = e->id;  // (?)

Смещение от начала объекта до `id` будет разное для Elf / ElfArcher. В зависимости от типа должен выбираться тот или иной метод, и это похоже на использование виртуальных методов классов по таблце вирт.методов. Поэтому тут при наследовании и использован такой механизм. Фактически, мы обращаясь к виртуальному базовому классу обращаемся к служебной виртуальной функции, которая выдает правильное смещение базового класса относительно текущего класса, из которого вызывается метод:

    unitid id = e->__getUnitPtr__()->id;

*) в зависимости от компилятора это может быть не виртуальный метод, а просто запись в таблице виртуальных методов

### <a id='toc1_3_2_'></a>[Кто вызывает конструктор виртуального базового класса?](#toc0_)

Пример:
```
struct unitid {
    explicit unitid( int i = 0 ) : id(i) {}
    int id;
};
struct Unit {
    Unit(unitid id, int health_points);
};
struct Elf: virtual Unit {
    explicit Elf(unitid id)
        : Unit(id, 100) {}
};
struct Archer: virtual Unit {
    explicit Archer(unitid id)
        : Unit(id, 120) {}
};
struct ElfArcher: Elf, Archer {
    explicit ElfArcher(unitid id)
        : Unit(id, 150)
        , Elf(id)
        , Archer(id) {}
};
```

Сколько `hp` должно быть у экземпляров `ElfArcher`? Как вариант мы могли бы указать например, первый базовый класс, или второй. Но по сути, это должно определяться логикой самого класса.

Поэтому принято, что за вызов конструктора базового виртуального класса отвечает самый нижний класс иерархии - в нем нужно прописывать нужные параметры такого вызова (несмотря на то, что это не наш непосредственный родитель). Таким образом убирается неоднозначность.
- при создании `ElfArcher` будет 1 раз вызван `Unit(id, 150)`
- при этом при вызове инициирующих `Elf(id)` и `Archer(id)` их соответствующие `Unit(...)` вызываться не будут

*) конструкторы тут вызываются явно, т.к. нет конструкторов по-умолчанию, а `explicit` они, чтобы нельзя было неявно привести число 10 к типу `Elf`, например

**Задача**

Предположим, что при разработке игры вам потребовался юнит "Челмедведосвин" (ManBearPig).
Завершите разработку иерархии классов и реализуйте необходимые конструкторы.
PS: В данном игровом контексте предполагается, что человек — не животное.

    /* этот класс уже определён выше
    struct Unit  
    {
        explicit Unit(size_t id) 
            : id_(id) 
        {}

        size_t id() const { return id_; }

    private:
        size_t id_;
    };
    */
    // базовый класс для животных
    struct Animal: virtual Unit
    {
        // name хранит название животного
        // "bear" для медведя
        // "pig" для свиньи
        // const std::string &s === std::string const &s
        Animal(std::string const & name, size_t id) 
            : name_(std::move(name))
            , Unit(id) 
        {}

        std::string const& name() const { return name_; }
    private:
        std::string name_;
    };

    // класс для человека
    struct Man: virtual Unit 
    {
        explicit Man(size_t id) 
            : Unit(id)
        {}
    };

    // класс для медведя
    struct Bear: Animal
    {
        explicit Bear(size_t id) 
            : Unit(id), Animal("bear", id)
        {}
    };

    // класс для свиньи
    struct Pig: Animal
    {
        explicit Pig(size_t id)
            : Unit(id), Animal("pig", id)
        {}
    };

    // класс для челмедведосвина
    struct ManBearPig: Man, Bear, Pig
    {
        ManBearPig(size_t id)
            : Unit(id)
            , Man(id)
            , Bear(id)
            , Pig(id)
        {}
    };

Так и не понял, зачем в Bear и Pig вызывать конструктор Unit() если он есть в Animal и он виртуальный, один на всех... Но без этого даже не компилируется

## <a id='toc1_4_'></a>[Выводы](#toc0_)

- Не используйте множественное наследование для наследования реализации.
- Используйте концепцию интерфейсов (классы без реализаций и членов данных).
- Помните о неприятностях, связанных с множественным наследованием.
- Хорошо подумайте перед тем, как использовать виртуальное наследование.
- Помните о неприятностях, связанных с виртуальным наследованием.

# <a id='toc2_'></a>[Преобразование типов в стиле C++](#toc0_)

## <a id='toc2_1_'></a>[Преобразование в стиле C](#toc0_)

Нет классов и шаблонов, поэтому оператор преобразования типов только 1 и этого более чем достаточно:

    int a = 2;
    int b = 3;

    double size = ((double) a) / b * 100;

    void * data = malloc(sizeof(double) * int(size));

    double * array = (double *) data;

    char * bytes = (char *) array;

*) оператор один `()`, но 2 допустимых синтаксиса: `int(var) === (int)var`. Первый вариант еще называют функциональным стилем - похож на выхов функции.

## <a id='toc2_2_'></a>[static_cast](#toc0_)

Служит для преобразований связанных типов:
- Стандартные преобрзования (к ним относят такие, которые также могут происходить неявно)
    - Преобразования числовых типов

    `double s = static_cast<double>(2) / 3 * 100`   
    `s  = static_cast<int>(d)`
    - Указатель/ссылка на производный класс **в** указатель/ссылку на базовый класс (связанные типы в смысле наследования) **upcasting** (вверх по наследованию)
    - `T*` (любой указатель) в `void*`
- Явное (пользовательское приведение типа)
    - пользовательское приведение типа задается через *констуктор 1 аргумента + оператор приведения типов*

    `Person p = static_cast<Person>("Ivan")`
    (в `Person` д.б. конструктор 1 арг. `Person(std::string const & name) { ... }` + оператор `operator Person() const { return Person { name }; }`)

- Обратные к стандратным преобразования
    - Указатель/ссылка на базовый класс **в** указатель/ссылку на производный класс (преобразование вниз по наследованию, **downcast**)
        - при этом проверок, что указатель на базовый класс валидный - не происходит, это должен делать программист
    - `void*` в `T*` (любой указатель)

- Любой тип к типу `void`
    - иногда полезно 
        - например, в некотором шаблоне мы заранее не знаем, какой будет тип, но шаблон сможет работать даже если там окажется `void`)
        - иногда к `void` приводят некоторые неиспользуемые аргументы функций, чтобы явно показать, что они не используются, и избежать предупреждений компилятора, но вообще - редкость

*) называется **статический** т.к. происходит на этапе компиляции (компилятор проверяет связанность типов и по ним пересчитывает указатели (но, конечно, не проверяет, что по указателю действительно расположены данные того типа, которого надо))

Нужно помнить три момента (когда `static_cast` обязательны):

1. **Upcast** возможен без оператора преобразования, **downcast** - нет;

2. Неявно можно привестись к указателю на `void*`, от указателя на `void*` к любому другому - нет;

3. explicit - конструкторы запрещнают неявное преобразование (например, нельзя написать String s = "foo bar", но можно BigInt bi = 13, поэтому типы с конструкторами `explicit` приводятся только через `static_cast`;

**Задача**

Выберите все случаи, где преобразование static_cast<> в правой части обязательно.


    struct Number {};

    struct BigInt : Number
    {
        BigInt(int x);
    };

    struct String
    {
        explicit String(char const * s);
    };

    int main() {
        int a = static_cast<int>(3.5);

        double b = static_cast<double>(7);
        BigInt c = static_cast<BigInt>(100500);
        String d = static_cast<String>("Stepik");   // без static_cast никак (explicit)
        Number * e = static_cast<Number *>(&с);
        BigInt * f = static_cast<BigInt *>(e);      // без static_cast никак (downcast)
        void * g = static_cast<void *>(f);
        BigInt * h = static_cast<BigInt *>(g);      // без static_cast никак (из void*)

        return 0;
    }

## <a id='toc2_3_'></a>[const_cast](#toc0_)

Служит для снятия/добавления константности

Обычно, добавление константности - это неявное преобразование (передавая неконстантные ссылки в функцию с константным аргументом получим такое неявное очевидное преобразование, это ОК)

Зачем надо бывает снимать константность: если хочется изменить значение по константной ссылке, то `const_cast` сделает это, например так, но зачем?

    void foo(double const& d) {
        const_cast<double &>(d) = 10;
    }

Использование `const_cast` - признак плохого дизайна.
- это еще и небезопасно. Если константная ссылка ссылается также на константу, то ее изменение таким вот образом - это UB

Кроме редких исключений:

    T & operator[](size_t i) {
        return const_cast<T &>(
            const_cast<Vector const &>(*this)[i]
            // или static_cast<Vector const &>(*this)[i]
        );
    }

    T const & operator[](size_t i) const {
        assert(i < size_);
        return data_[i];
    }

Тут приводится пример реализации парных методов в классах, когда мы делаем две версии метода (константная и неконстантная). Часто так реализуют пары геттеры/сеттеры. В этом случае имеется проблема дублирования кода. Чтобы не дублировать код, мы можем реализовать константный вариант через неконстантный, или наоборот - неконстантный через константный. Первый вариант является методически неверным - использовать для метода, который не должен изменять данные основу в виде метода, который может это делать - не корректно. Поэтому должен использоваться второй вариант, как в примере выше (через константный геттер делаем с помощью снятия константности `const_cast` неконстантный сеттер для вектора):

- `const_cast<Vector const &>(*this)` добавляет константность разыменованной ссылке на вектор (`this` это экземпляр класса вектора)
    - `Vector` - это сам контейнер, тот класс, в котором лежат объекты типа T. Чтобы сработал константный оператор квадратных скобок, он должен быть вызван у константного объекта (`const Vector`). Поэтому сначала преобразуется к константному объекту текущий экземпляр вектора (`*this`), затем, получив значение `T const &`, преобразуем его к `T &`.
    - в этом месте ничего не мешает нам использовать просто `static_cast<Vector const &>`, константность точно также добавится и константный `const & operator[]` с ней нормально отработает
- наш геттер `[i]` возвращает константную ссылку на элемент вектора, поэтому второй `const_cast<T &>(...[i])` эту константность снимает и мы можем установить туда новое значение
- **т.е. фактически мы только снимаем константность которую сами же тут же и устанавливали, поэтому такой подход можно считать вполне корректным в отличие от произвольного снятия константности, которое скорее всего для сторонних пользователей, считающих, что если что-то константа, то это константа, будет неожиданностью**
- в данном конкретном случае, ничего не мешает нам продублировать 1 строчку кода константного геттера в сеттере (`assert(i < size_);`), и дальше обращаться сразу к `data_[i]`, это было бы лучше стремного паттерна двойного `const_cast`. Но если геттер объемный, с большим числом проверок и т.п., то данный вариант - рабочий.

**Задача**

Где преобразование const_cast<> в правой части обязательно:

    int main() {
        int a = 27;
        int const b = 412;
        int * pa = &a;

        int const c = const_cast<int const &>(a);
        int d = const_cast<int &>(b);
        int const * p1 = const_cast<int const *>(pa);
        int * const * p2 = const_cast<int * const *>(&pa);
        int const * const * p4 = const_cast<int const * const *>(&pa);
        
        int const ** p3 = const_cast<int const **>(&pa);    // только тут

        return 0;
    }

В данном примере остальные типы, кроме выделенного, привелись бы и неявно. Единственный случай, когда нужно явное преобразование типа - это **константный указатель на указатель**. Неявное приведение такого типа в С++ запрещено, т.к. позволило бы изменять константные данные:

    int const i = 1;
    int * p = 0;
    // p = &i;  не скомпилируется,  т.к. это преобразование int const * -> int *

    // В этой строке происходит запрещённое преобразование int ** -> int const **
    int const ** pp = &p;  // теперь *pp указывает на переменную p

    // следующая строка скомпилируется, т.к. *pp имеет тип int const*
    *pp = &i; // это соответствует p = &i
    *p = 2; // изменяем значение переменной i 

Говоря точнее, стандартом разрешены (неявные) преобразования от T* к T const*, но не T** к T const**.


## <a id='toc2_4_'></a>[reinterpret_cast](#toc0_)

Служит для преобразования указателей и ссылок на несвязанные типы.

Например, есть массив даблов, мы хотим преобразовать его к массиву чаров (рассматривать данные, которые представляет массив типа дабл, как данные, которые представляют массив типа чар, например, нам нужно совсем низкоуровневое представление объекта, с которым мы хотим что-то сделать, например, послать по сети).

Пусть у нас есть некоторый API, который умеет `send`/`receive` наборы байт:

    // указатель на начало данны, количество байт для пересылки, возвращает ничего
    void send(char const * data, size_t length); 

    // возвращает указатель на начало принятых данных, в аргументе передается переменная, в которую запишет количество принятых байт (т.к.не бывает функций, возвращающих больше 1 значения)
    char * receive(size_t * length); 

- посылаем
```
    double * m = static_cast<double *>(malloc(sizeof(double) * 100));
    ...// инициализация m
    char * m = reintpret_cast<char*>(m);
    send(mc, sizeof(double) * 100);
```
- в посылке мы использовали сишный `malloc` для примера. Конечно корректный вариант был бы `new[]`
    - `static_cast` может и поменять значение адреса по указателю, если у нас сложное наследование и целевой тип указателя относится к структуре, сидящей внутри родительского класса по некоторому, отличному от родительского, адресу
    - `reinterpret_cast` никогда не меняет значение адреса указателя, только его тип!
- принимаем
```
    size_t length = 0;
    double * m = reinterpret_cast<double *>(receive(&length))
    length = length / sizeof(double)    // char -> double
```

В целом `reinterpret_cast` обычно может использоваться в низкоуровневых частях кодовой базы проекта (сериализация, десериализация и т.п.), в высокоуровневых частях обычно в нем нет необходимости.

Может использоваться также для преобразования **указателя в число и обратно**

    size_t ms = rinterp_cast<size_t>(m);

- например, мы хотим сами что-то сделать с некоторым адресом не используя встроеннуя арифметику указателей (конечно, это не считается особо безопасным подходом)

`reinterpret_cast` может быть (почти всегда) заменен двойным использованием `static_cast` через `void*`:

    reinterpret_cast<Type*>(value_to_cast) => static_cast<Type*>(static_cast<void*>(value_to_cast))


## <a id='toc2_5_'></a>[Границы применимости преобразования в стиле C](#toc0_)

В С++ доступны оба стиля преобразования указателей/ссылок. Преобразования в стиле С++ всегда можно заменить на преобразования в стиле C (но не наоборот)

- static_cast
- const_cast
- reinterpret_cast

Преобразования в стиле С более суровая штука, с ними надо осторожнее. Например, есть код где ожидаются связанные наследованием типы и они приводятся `static_cast`. Если использовать стиль С, то когда придут вдруг несвязанные типы, он их прекрасно преобразует и можем получить UB вместо сообщения об ошибке.

Когда оправдано использовать стиль С в С++:
- очевидные преобразования встроенных типов
- преобразования указателей на явные типы

Когда не стоит использовать стиль С:
- с пользовательскими типами и указателями на них
- в шаблонах

В `gcc` можно использовать директиву `-Wold-style-cast` чтобы отследить где используюется стиль С.

В большой кодовой базе при использовании приведения типов в стиле С++ вы улучшаете последующую поддержку своего кода, так как поиск мест приведения типов получается однозначным. В большой кодовой базе количество строк содержащее `(int) (double) int()` и т.п. будет в большом количестве объявлений функций и классов, что сильно увеличивает время поиска мест конкретно приведения типов.

## <a id='toc2_6_'></a>[Когда преобразование в стиле C приводит к ошибке](#toc0_)

    // abc.h
    struct A { int a; };
    struct B {};
    struct C : A, B {};

В памяти это будет:

    |A  |B  |C  |
    ^   ^
    C*  B*

Приведение от `B*` к `C*`

    #include "abc.h"

    C * foo(B * b) {
        return (C *)b;
    }

Если будет добавлен `#include "abc.h"`, т.е. компилятору будут известны определения классов A,B,C, то компилятор в этом месте заменит `(C *)b` на `static_cast<C>(b)` и все будет работать ожидаемым образом.

    struct A; struct B; struct C; 

    C * foo(B * b) {
        return (C *)b;
    }

Для возвращения адреса определение структуры компилятору знать необязательно, а для возвращения по значению обязательно.

Поэтому если классы будут только объявлены (forward declaration), то это позволит скомпилировать код (т.к. мы к полям и методам классов не обращаемся). Но т.к. компилятор не будет знать о внутреннем устройстве классов, то вместо `(C *)b` он использует `reinterpret_cast<C>(b)`, которые не сдвигает указатель и мы получим в чистом виде UB (указатель будет на адрес `B*`, но иметь тип `C`)

# <a id='toc3_'></a>[Информация о типах времени выполнения](#toc0_)

## <a id='toc3_1_'></a>[Run Time Type Information (RTTI)](#toc0_)

В С++ этот механизм состоит из 2 компонент:
- оператор `typeid` и `std::type_info`
- оператор `dynamic_cast`


## <a id='toc3_2_'></a>[Использование typeid и type_info](#toc0_)

- `type_info` это класс, объявленный в системных заголовках `<typeinfo>`
- содержит информацию о типе
- имеет методы: `==`, `!=`, `name` (получение имени типа в виде `char*`), `before` (для установки отношения порядка на типах, конкретный порядок опред. компилятором), `hash_code` (нововведение стандарта, для организации хеш-таблиц из типов данных)
    - о самом типе информации очень мало, кроме названия и сравнения/порядка в данном классе ничего нет
- нет публичных конструкторов и оператора присваивания
    - сами мы не можем создавать/присваивать этот тип
- можно получить ссылку на `type_info`, соответствующий значению или типу, при помощи оператора `typeid`

*) `before` может использоваться, например, чтобы поместить множество типов в список/дерево и искать в нем нужный тип быстрее, чем за O(n)

Пример:

    struct Unit {
        virtual ~Unit() {};
    };

    struct Elf : Unit { };

    int main() {

        Elf e;
        Unit & ur = e;
        Unit * up = &e;
                                                            // virt     NO virt
        std::cout << typeid(ur).name() << "\n";             // 3Elf     4Unit
        std::cout << typeid(*up).name() << "\n";            // 3Elf     4Unit
        std::cout << typeid(up).name() << "\n";             // P4Unit   P4Unit
        std::cout << typeid(Elf).name() << "\n";            // 3Elf     3Elf
        std::cout << (typeid(ur) == typeid(Elf)) << "\n";   // 1        0

        return 0;
    }

С полиморфными (`virtual`) классами `typeid` умеет возвращать производный класс по базовому (если обращаться по ссылке или разыменованному указателю, с непосредственно указатаелем - возвращается именно его тип). 

**Задача**

В первой части курса была следующая задача.

Вам требуется реализовать функцию, которая принимает на вход два указателя на базовый класс Expression, и возвращает true, если оба указателя указывают на самом деле на объекты одного и того же класса, и false в противном случае (т.е. если оба указателя указывают на BinaryOperation, то возвращается true, а если один из них указывает на Number, а второй на BinaryOperation, то false).

Реализуйте её при помощи `typeid`.

    #include <typeinfo>

    bool check_equals(Expression const *left, Expression const *right)
    {
        return (typeid(*left) == typeid(*right));
    }

*) Очевидно, но иногда забывают: Standard does not guarantee unique names for types. Сравнивать `name()` низзя.

## <a id='toc3_3_'></a>[dynamic_cast](#toc0_)

Преобразования с проверкой типа времени выполнения

    Unit * u = (rand() % 2) ? new Elf() : new Dwarf();
    ... // работаем с объектом через указатель на базовый класс, не важно это Elf или Dwarf
    
    // если вдруг понадобиться поработать с ним непосредственно как с Elf или Dwarf, то
    if (Elf * e = dynamic_cast<Elf *>(u))
        ... // работаем с Elf, если удалось кастовать к нему
    else if (Dwarf * d = dynamic_cast<Dwarf *>(u))
        ... // работаем с Dwarf, если удалось кастовать к нему
    else
        ... // если что-то пошло не так...

В случае ошибки приведения типа dynamic_cast возвращает нулевой указатель, поэтому его тут можно использовать в уловных переходах

Особенности:
- не может быть заменен преобразованием в стиле С
- требует наличие полиморфности (таблиц виртуальных методов, по которым он может вывести нужный тип и найти правильный указатель на нужное место) 

Вопросы:
- почему стоит избегать RTTI
    - способствует формированию лапши в коде (если много альтернативных типов)
    - более низкая производительность (dynamic_cast делает проверку типов в рантайме и работает медленнее static_cast, который типы не проверяет)
- вопрос на сообразительность, что возвращает вызов `dynamic_cast<void *>(u)`
    - кастует к самому нижнему классу иерархии? Почти угадал, вернее не почти, а все верно. 
    - Предположим, что мы выделили в памяти объект какого-нибудь сложного класса с множественным наследованием. Тогда, как мы уже знаем, указатели разных типов на один и тот же объект могут хранить разные адреса. Оператор приведение `dynamic_cast<void *>` для некоторого объекта возвращает самый маленький адрес из всех возможных адресов для этого объекта, т.е. адрес его первого байта.

**Задача**

Предположим, что вам даётся два указателя на объект некоторого сложного полиморфного класса, который использует множественное наследование. Оба указателя имеют один и тот же тип, но могут ссылаться на разные однотипные базы внутри одного объекта.

Напишите шаблонную функцию, которая по двум однотипным указателям проверяет, указывают ли они на один объект.

    template<class T>
    bool isSameObject(T const * p, T const * q)
    {
        return (dynamic_cast<void const *>(p) == dynamic_cast<void const *>(q));
    }

Важно: указатели приходят константные, если их попробовать привести к `<void *>` это будет фактически снятие константности, компилятор не позволит сделать это таким образом, без `const_cast`:

    error: cannot dynamic_cast ‘p’ (of type ‘const struct Unit*’) to type ‘void*’ (conversion casts away constness)

## <a id='toc3_4_'></a>[Когда можно избежать использования RTTI](#toc0_)

(длинная кописта)

В некоторых случаях можно избежать использования RTTI. В этом примере показывается, как можно избежать использования RTTI при реализации мультиметодов. Мультиметод — это обобщение понятия виртуального метода. При вызове виртуального метода выбор конкретной реализации зависит от типа одного объекта (от типа того объекта, у которого этот виртуальный метод вызывается). При вызове мультиметода выбор реализации может зависеть от типов нескольких объектов.

Рассмотрим следующий пример. У класса `Shape` (фигура) есть три наследника: Rectangle (прямоугольник), `Triangle` (треугольник) и `Circle` (круг). Мы хотим реализовать функцию `is_intersect`, которая проверяет две фигуры на пересечение. Предположим, что у нас уже реализованы 6 необходимых функций, которые умеют пересекать все комбинации фигур (круг-круг, круг-прямоугольник,...). 
Как реализовать функцию `is_intersect`, которая принимает фигуры по указателю на базовый класс?

Можно было бы написать что-то подобное.

    // реализация самой функции
    bool is_intersect(Shape * a, Shape * b) 
    { 
        if (Rectangle * ra = dynamic_cast<Rectangle *>(a))
        {
            if (Rectangle * rb = dynamic_cast<Rectangle *>(b))
            {
                ...// вызываем реализацию для двух прямоугольников
            }
            else if (Triangle * tb = dynamic_cast<Triangle *>(b))
            {
                ...// вызываем реализацию для прямоугольника и треугольника
            }
            ....
        }
        else if (Triangle * ta = dynamic_cast<Triangle *>(a)) 
        ....
    }

Однако такой код нельзя назвать хорошим: реализация этой функции будет состоять из 9 веток if-else, кроме того нам потребуется сделать множество dynamic_cast.

В следующем решении этой задачи используется так называемая "**двойная диспетчеризация**", которая позволяет избежать использования `dynamic_cast`.

    // объявляем наследников Shape
    struct Rectangle;
    struct Triangle;
    struct Circle;

    // все 6 вариантов функций пересечения (имитация, ничего не делает)
    bool is_intersect(Rectangle * a, Rectangle * b) { std::cout << "R x R\n"; return true; }
    bool is_intersect(Rectangle * a, Triangle  * b) { std::cout << "R x T\n"; return true; }
    bool is_intersect(Rectangle * a, Circle    * b) { std::cout << "R x C\n"; return true; }
    bool is_intersect(Triangle  * a, Triangle  * b) { std::cout << "T x T\n"; return true; }
    bool is_intersect(Triangle  * a, Circle    * b) { std::cout << "T x C\n"; return true; }
    bool is_intersect(Circle    * a, Circle    * b) { std::cout << "C x C\n"; return true; }

    // базовый класс фигуры
    struct Shape 
    { 
        virtual ~Shape() {} 
        virtual bool intersect( Rectangle * r ) = 0;
        virtual bool intersect( Triangle  * t ) = 0;
        virtual bool intersect( Circle    * c ) = 0;
        virtual bool intersect( Shape     * s ) = 0;
    };

    // прямоугольник
    struct Rectangle : Shape 
    {
        bool intersect( Rectangle * r ) { return is_intersect(this, r); }
        bool intersect( Triangle  * t ) { return is_intersect(this, t); }
        bool intersect( Circle    * c ) { return is_intersect(this, c); }
        bool intersect( Shape     * s ) { 
            return s->intersect(this); 
        }
    };

    // треугольник
    struct Triangle : Shape 
    {
        bool intersect( Rectangle * r ) { return is_intersect(r, this); }
        bool intersect( Triangle  * t ) { return is_intersect(t, this); }
        bool intersect( Circle    * c ) { return is_intersect(this, c); }
        bool intersect( Shape     * s ) { 
            return s->intersect(this); 
        }
    };

    // круг
    struct Circle : Shape 
    {
        bool intersect( Rectangle * r ) { return is_intersect(r, this); }
        bool intersect( Triangle  * t ) { return is_intersect(t, this); }
        bool intersect( Circle    * c ) { return is_intersect(c, this); }
        bool intersect( Shape     * s ) { 
            return s->intersect(this); 
        }
    };

    // реализация самой функции
    bool is_intersect(Shape * a, Shape * b) 
    { 
        return a->intersect(b); 
    }

Разберитесь, как устроено это решение: каким образом после двух последовательных вызовов метода intersect (сначала внутри функции `is_intersect`, а потом внутри реализации `intersect(Shape*))` мы попадаем в метод `intersect`, в котором типы обоих аргументов уже известны, и в котором мы можем вызвать `is_intersect` для конкретных типов.

Замечание: этот подход очень похож на паттерн посетитель (visitor), который встречался в задачах в первой части курса.

Тогда в мейне:

    int main() {
        Shape * a = new Circle;
        Shape * b = new Triangle;
        is_intersect(a, b);
    }

По таблице вирт. методов в `a` будет найден `Circle.intersect()`, по таблице вирт. методов `b` будет выбран его вариант `Circle.intersect(t, this)`. В то же время с `dynamic_cast` к таблицам вирт. методов будет от 2 обращений (в лучшем случае, если сматчится на первом if-else), до 6 (если сматчится на последнем). То есть, такой вариант в среднем быстрее использования `dynamic_cast`.

# <a id='toc4_'></a>[Указатели на функции](#toc0_)

## <a id='toc4_1_'></a>[Указатели на функции](#toc0_)

Кроме указателей на значения С++ присутствуют три особенных типа указателей:
- указатели на функции (наследство С)
- указатели на методы
- указатели на поля классов

*) арх. фон Неймана, когда код и данные лежат в одном адресном пространстве, позволяет использовать указатели на функции (сегменты кода)

Указатели на функции (и методы) используются для:
1. Параметриации алгоритомв
2. Обратных вызовов (callback)
3. Подписки на события (шаблон Listener)
4. Создания очередей событий / заданий

## <a id='toc4_2_'></a>[Указатели на функции: параметризация алгоритмов](#toc0_)

`qsort` из стандратных библиотек С/С++ устроена примерно так:

    void qsort(void* base. size_t num, size_t size, int (*compar) (void const*, void const*));

- мы ничего не знаем про тип сортируемых данных (`void* base`)
- передаем указатель на начало массива, число элементов, размер элемента, указатель на функцию-компаратор с сигнатурой ее аргументов
- в случае указателя на функцию в скобках вместо типа указывается имя функции: `*compar`
- код функции компаратора:

```
    int doublecmp(void const * a, void const * b) {
        double da = *static_cast<double const *>(a);
        double db = *static_cast<double const *>(b);
        if (da < db) return -1;
        if (da > db) return 1;
        return 0;
    }
```

- обертка для `qsort` для сортировки массивов типа double, передаваемых через указатели на начало и конец:
```
    void sort(double * p, double * q) {
        qsort(p, q - p, sizeof(double), &doublecmp);

        // как вариант, можно без &, т.к. имя функции всегда неявно приводится к указателю на функцию
        //qsort(p, q - p, sizeof(double), doublecmp);
    }
```

Но вообще, передавать что-то как void* плохо. Не будет вообще никаких проверок типов.

**Сделаем этот пример типобезопасным**

    void sort(int * p, int * q, bool (*cmp)(int, int)) {
        for (int * m = q; m != p; -- m)
            for (int * r = p; r + 1 < m; ++r)
                if (cmp(*(r + 1), *r))
                    swap(*r, *(r + 1));
    }

    bool less (int a, int b) { return a < b; }
    
    bool greater (int a, int b) { return a > b; }
    
    void sort(int * p, int * q, bool asc = true) {
        sort(p, q, asc ? &less : &greater);
    }

Теперь ее можно зашаблонить и использовать для любых типов.

## <a id='toc4_3_'></a>[О полезности typedef](#toc0_)

Что здесь объявлено?

    char * (*func(int, int)) (int, int, int *, float);

Это функция двух целочисленных переменных, возвращающая указатель на функцию, которая возвращает char* и имеет собственный список переменных вида (int, int, int *, float)

1) `func (int, int)` - func - это фукнция, принимающая 2 инта

2) `* func (int, int)`  - возвращающая указатель

3) `( * func (int, int)) (int, int, int*, float)`  - на функцию, принимающую указанный список параметров

4) `char * ( * func (int, int)) (int, int, int*, float)` - и возвращающую указатель на char

Как стоило это написать:

    typedef char * (*MyFunction) (int, int, int *, float);
    MyFunction func(int, int);

- или
```
    typedef char* Function(int, int, int*, float);
    typedef Function* MyFunction(int, int);
```

**Задача**
При помощи typedef определите тип ComplexFunction, который является указателем на функцию, которая принимает int и указатель на функцию, принимающую double и возвращающую int, и возвращает указатель на функцию, которая принимает char const * и возвращает int *.

    //typedef ComplexFunction
    typedef int (*func) (double);
    typedef int * (*Func) (char const *);
    typedef Func (*ComplexFunction) (int, func);


## <a id='toc4_4_'></a>[Шаблон проектирования Listener](#toc0_)

Указатели на функции могут потребоваться, например, при реализации шаблона проектирования "наблюдатель" ("listener").

Шаблон "наблюдатель" можно объяснить на следующем примере. Предположим, что мы программируем графический интерфейс. Как нам связать объект-кнопку и действие, которое произойдёт, когда на кнопку нажмут? Можно для каждой кнопки писать отдельный класс, но это приведёт к дублированию кода. Вместо этого можно написать общий класс для кнопок и настраивать объекты этого класса под каждую конкретную кнопку интерфейса. Так для каждого объекта можно будет задавать, например, положение, надпись на кнопке и соответствующее действие. Для того, чтобы задавать действие, мы можем добавить в его интерфейс метод subscribe, который позволяет подписаться на событие "кнопка нажата".

Например, в терминах ООП это могло бы выглядеть так.
    
    struct Button;

    // интерфейс, который реализуют все классы,
    // объекты которых хотят подписаться на событие
    // "кнопка нажата"
    struct ButtonListener 
    {
        // метод, который будет вызываться, когда
        // происходит событие "кнопка нажата"
        virtual void onButtonClick(Button * b, bool down) = 0;
        virtual ~ButtonListener(){}
    };

    // общий класс для кнопок
    struct Button 
    {
        // метод позволяет подписать объект класса,
        // реализующего интерфейс ButtonListener
        // на событие "кнопка нажата"
        void subscribe( ButtonListener * bl );
        ...
    };

Этот же шаблон проектирования можно реализовать без наследования, виртуальных методов и интерфейсов при помощи указателей на функции.

Решение с помощью указателей на функции (а также как вариант кода чисто на С, без классов):
    
    struct Button;

    // тип указателя на функцию, которая будет вызываться,
    // когда событие "кнопка нажата"
    typedef void (*ButtonProc)(Button *, bool, void *);

    // общий класс для кнопок
    struct Button 
    {
        // метод позволяет подписать объект функцию
        // типа ButtonProc на событие "кнопка нажата"
        void subscribe( ButtonProc bp, void * arg );
        ...
    };



## <a id='toc4_5_'></a>[Пример для самостоятельных экспериментов](#toc0_)

Один из примеров, на которых можно увидеть, как применяются указатели на функции — это использование Linux библиотеки pthread для написания многопоточных приложений. Библиотека `pthread` позволяет создавать потоки в POSIX системах.

Ниже приведён пример кода с использованием библиотеки pthread, который вычисляет 
$\int_{-5}^{5} e^{x\cdot sin(x)}dx$, разбивая интегрирование на несколько потоков


    #include <iostream>
    #include <cmath>

    // подключаем библиотеку pthread
    #include <pthread.h>

    // функция создания потоков имеет следующую сигнатуру
    //int pthread_create(pthread_t *thread, const pthread_attr_t *attr,
    //                   void *(*start_routine) (void *), void *arg);

    // структура для хранения данных о задаче интегрирования
    struct Task
    {
        // функция
        double (*f) (double);

        // начальная точка отрезка
        double start;

        // конечная точка отрезка
        double end;

        // шаг интегрирования
        double eps;

        // куда записать результат
        double * result;
    };

    // функция потока
    void * thread_fun(void * data)
    {
        // получаем задачу
        Task & t = *static_cast<Task *>(data);

        // переменная для результата
        double res = 0;
        for (double x = t.start; x < t.end; x += t.eps)
            res += t.f(x + t.eps / 2) * t.eps;

        *t.result = res;
        return 0;
    }

    // функция, которую будем интегрировать
    double function(double x)
    {
        return exp(x * sin(x));
    }

    int main( )
    {
        // количество потоков
        int const THREAD_COUNT = 8;

        // массив, в который будут записаны результаты
        double results[THREAD_COUNT] = {};

        // задачи для потоков
        Task tasks[THREAD_COUNT] = {};

        // пределы интегрирования
        double const start = -5;
        double const end   =  5;

        // заполняем задачи
        double const taskLen = (end - start) / THREAD_COUNT;
        for (int i = 0; i != THREAD_COUNT; ++i)
        {
            tasks[i].f      = &function;
            tasks[i].start  = start + taskLen * i;
            tasks[i].end    = tasks[i].start + taskLen;
            tasks[i].eps    = 1e-7;
            tasks[i].result = &results[i];
        }

        // создаём дескрипторы потоков
        pthread_t threads[THREAD_COUNT];

        // создаём и запускаем потоки
        for (size_t i = 0; i != THREAD_COUNT; ++i)
            pthread_create(&threads[i], 0, &thread_fun, &tasks[i]);

        // дожидаемся завершения потоков
        for (size_t i = 0; i != THREAD_COUNT; ++i)
            pthread_join(threads[i], 0);

        // складываем результаты
        double res = 0;
        for (size_t i = 0; i != THREAD_COUNT; ++i)
            res += results[i];

        // выводим результат
        std::cout << "Result: " << res << std::endl;

        return 0;
    }


При компиляции нужно указать, что используется библиотека pthread (это нужно для линковщика).
    
    g++ integrate.cpp -O2 -o integrate -pthread

Попробуйте поэкспериментировать с этим кодом.
Если захотите сравнить время различных модификаций, то самый простой способ это сделать в Linux — команда time.

    time ./integrate

## <a id='toc4_6_'></a>[Пример для самостоятельных экспериментов 2](#toc0_)

Другой пример, на котором можно увидеть, как применяются указатели на функции — это библиотека Expat для парсинга XML. Библиотека Expat реализует так называемый SAX-парсер. Когда в процессе разбора XML встречается, например, новый тег, то вызывается пользовательская функция для обработки этого события. Для того, чтобы установить свою функцию обработки, в библиотеку нужно передать указатель на эту функцию.

Ниже приведён пример кода с использованием библиотеки Expat, который разбирает XML и выводит содержимое тегов "title".

    #include <iostream>
    #include <fstream>

    #include <string>
    #include <vector>

    #include <expat.h>

    using std::string;

    // структура для хранения пользовательских данных
    struct MyData 
    {
        // заголовки новостей
        std::vector<string> news;
        
        // текущий открытый тег
        string openTag;

        // текст внутри тега title
        string title;
    };

    // вызывается для открывающего тега
    // el - имя тега
    // attr - аттрибуты тега
    void start( void * data, const char * el, const char ** attr )
    {
        MyData & d = *static_cast<MyData *>(data);

        // сохраняем последний открытый тег
        d.openTag = el;
    }

    // вызывается для закрывающего тега
    // el - имя тега
    void end( void * data, const char * el )
    {
        MyData & d = *static_cast<MyData *>(data);
        if ( d.openTag == "title" )
        {
            // сохраняем заголовок в векторе
            d.news.push_back(d.title);

            // очищаем title
            d.title = "";    
        }
    }

    // вызывается для текста внутри тегов
    void text(void * data, const char *s, int len)
    {
        MyData & d = *static_cast<MyData *>(data);
    
        if (d.openTag == "title")
        {
            // добавиляем текст заголовка
            d.title += string(s, len);
        }
    }

    int main()
    {
        // создаём XML парсер
        XML_Parser p = XML_ParserCreate(0);

        // открываем файл с XML в бинарном режиме
        std::ifstream f("rss.xml", std::ifstream::binary);

        // обработка ошибки открытия файла
        if (!f.is_open())
        {
            std::cerr << "Can not open 'rss.xml' for reading\n";
            return 1;        
        }

        // устанавливаем указатель на функции, которые
        // будут вызывать при встрече открывающего тега 
        // (start) и закрывающего тега (end)
        XML_SetElementHandler(p, start, end);

        // устанавливаем указатель на функцию для обработки 
        // текста внутри XML тегов
        XML_SetCharacterDataHandler(p, text);

        // пользовательские данные, которые мы будем использовать
        // при парсинге XML
        MyData data;

        // устанавливаем указатель на пользовательские данные 
        XML_SetUserData(p, &data);
            
        // будем считывать по 1Кб за раз
        char buff[1024];
            
        // читаем данные из файла
        while(f.read(buff, sizeof(buff)))
        {
            // колличество считанных байтов
            size_t len = f.gcount();
            
            // парсим прочитанный кусочек
            if (!XML_Parse(p, buff, len, f.eof()))
            {
                // обработка ошибок при парсинге
                std::cerr << "Parser error at line " 
                        << XML_GetCurrentLineNumber(p) << ": "
                        << XML_ErrorString(XML_GetErrorCode(p))
                        << std::endl;    
                return 1;
            }        
        }                                                   

        // выводим все заголовки на стандартный вывод
        for (size_t i = 0; i != data.news.size(); ++i)
            std::cout << data.news[i] << std::endl;

        return 0;    
    }


Этот код можно использовать, например, для того, чтобы вывести все заголовки новостей из какого-нибудь RSS-потока. Например, можно скачать какой-нибудь RSS файл:

    wget "https://stackoverflow.com/feeds/tag?tagnames=c%2b%2b&sort=newest" -O rss.xml

(в этом примере скачиваются последние темы с сайта StackOverflow с тегом "c++").
Для того, чтобы скомпилировать этот код, вам потребуется установить библиотеку Expat.
В Ubuntu это можно сделать следующей командой:

    sudo apt install libexpat1-dev

Alt Linux:

    sudo apt-get install libexpat-devel

При компиляции нужно указать, что используется библиотека expat (это нужно для линковщика).

    g++ rss.cpp -o rss -lexpat

Попробуйте поэкспериментировать с этим кодом, например, улучшить его или написать свой парсер для какого-нибудь другого типа XML документов.

# <a id='toc5_'></a>[Указатели на методы и поля класса](#toc0_)

## <a id='toc5_1_'></a>[Указатели на методы: параметризация алгоритмов](#toc0_)

Метод отличается от функций тем, что у него есть неявный параметр `this`, соответствующий эземпляру объекта, для которого вызывается метод.

Поэтом для вызова метода по указателю нужен объект.

    struct Unit {
        virtual unsigned id() const;
        virtual unsigned hp() const;
    };

Определим тип указателя на метод класса (это удобнее С синтаксиса указателей на функции), назовем его UnitMethod. Его сигнатура должна полностью соответствовать методу, в т.ч. констанстность

    typedef unsigned (Unit::*UnitMethod) () const

    void sort (Unit * p, Unit * q, UnitMethod mtd) {
        for (Unit * m = q; m != p; --m)
            for (Unit * r = p; r + 1 < m; ++r)
                if ( (r->*mtd)() > ((r+1)->*mtd)() )
                    std::swap(*r, *(r+1));
    }

    sort(p, q, &Unit::hp);

Теперь можем сортировать юниты по id или hp.

Тут для вызова методов по указателю: берем указатель на объект `r`, по стрелке и разыменованному указателю на метод `*mtd` получаем `r->*mtd` метод объекта, а затем вызываем этот метод `(r->*mtd)()`

Итого, синтаксические **отличия вызова** метода по указателю от функции по указателю:
- метод из указателя нужно обязательно разыменовать в метод (функцию не надо)
- при передаче метода `&Unit::hp` в аргумент функции `&` обязательно (неявного приведения как с функциями не произойдет)

**Замечание**

На данный момент все использования `typedef` можно заменить на `using`, они считаются эквивалентными. К тому же, `using` можно использовать не только для определения псевдонима типа.

При помощи `using` пример можно записать как:

    using UnitMethod = unsigned int (Unit::*) () const;

Запись в такой форме может быть более удобной для восприятия, поскольку определяемый псевдоним находится в самом начале, а не скрыт внутри между звёздочками и скобочками

**Замечание**

Такой же синтаксис используется в qt для соединения сигналов и слотов:

    connect(sender, &Sender::valueChanged, receiver, &Receiver::updateValue );

## <a id='toc5_2_'></a>[Указатели на поля: параметризация алгоритмов](#toc0_)

Если указатели не методы похожи на указатели на функции, то указатели на поля похожи на просто указатели

Аналогично, для обращения к полю также нужен объект:

    struct Unit {
        unsigned id;
        unsigned hp;
    };

Определим тип указателя на поле класса

    typedef unsigned Unit::*UnitField

    void sort (Unit * p, Unit * q, UnitField f) {
        for (Unit * m = q; m != p; --m)
            for (Unit * r = p; r + 1 < m; ++r)
                if ( (r->*f) > ((r+1)->*f) )
                    std::swap(*r, *(r+1));
    }

    sort(p, q, &Unit::id);

**Замечание**

По аналогии с предыдущим замечанием про указатели на функции-члены классов, переписываем typedef на using:

    using UnitField = unsigned int Unit::*;

## <a id='toc5_3_'></a>[Резюме по синтаксису](#toc0_)

Указатели на методы и поля класса

    struct Unit {
        unsigned id() const;
        unsigned hp;
    };

    unsigned (Unit::*mtd)() const = &Unit::id;
    unsigned Unit::*fld           = &UNit::hp;

    Unit u;
    Unit * p = &u;

    (u.*mtd)() == (p->*mtd)();
    (u.*fld) == (p->*fld);

**Замечание**

`->*` и `.*` на самом деле это отдельные операторы, которые можно переопределить. Суть их конечно такая же как последовательное применение отдельных операторов, но строго говоря они сами по себе операторы языка.

## <a id='toc5_4_'></a>[Как такие указатели устроены?](#toc0_)

Что хранится в указателе на **функцию**:
- адрес функции в сегменте кода (по которому расположена точка входа в функцию)

Что хранится в указателе на **поле** класса:
- смещение поля от начала объекта (адрес объекта нам известе из правой части выражения `u->*field`, тип поля известен из определения объекта)

Что хранится в указателе на **метод**:
- если это обычный метод неполиморфного класса, то - адрес метода в сегменте кода (это такая же функция как и все остальные с т.з. компилятора)
- если же метод виртуальный, то тут будт храниться номер метода в таблице виртуальных методов
- если это метод базового класса через указатель на производный класс, то для его вызова может потребоваться еще и **смещение** данных базового класса в производном

## <a id='toc5_5_'></a>[Зачем нужно смещение?](#toc0_)

Пример:

    struct Elf {
        string secretName;
    }

    struct Archer {
        unsigned arrows() { return arrows_; }
        unsigned arrows_;
    }

    struct ElfArcher : Elf, Archer {};

    void foo() {
        ElfArcher ea;
        unsigned (ElfArcher::*m)() = &Archer::arrows;
        (ea.*m)();
    }

Указатель `m` на метод класса `ElfArcher`. Мы явно указываем, что это метод из базового класса `Archer` (этот метод уже скомпилирован к моменту этого объявления). Вызываем его через переменную типа `ElfArcher`. Как компилятор найдет адрес этого метода в сегменте кода?

- по вызову непосредственно `Archer::arrows()` все просто, в этом случае метод получает `this`, соответствующий `Archer` и по которому сразу лежат данные `arrows_`, которые метод считывает и возвращает
- при вызове через указатель/ссылку на `ElfArcher` в `this` унаследованному методу `Archer::arrows()` передается `this`, соответствующий `ElfArcher`, а эта структура начинается с поля `string secretName`. Методу нужно еще знать смещение данных, унаследованных от `Archer` в этой структуре
- в этом случае при присваивании `unsigned (ElfArcher::*m)() = &Archer::arrows` в указатель заносится данное смещение. Если потом в этот указатель кладется адрес другого метода, уже напосредственно класса `ElfArcher`, то ему будет установлено смещение 0.

## <a id='toc5_6_'></a>[Важные моменты](#toc0_)

- Использование неинициалированных указателей на функции и методы влечет UB.
- Для использования указателей на методы и поля классов нужны экземпляры этих классов.
- Указатели на методы и поля класса ни к чему не приводятся, это не обычные указатели, это отдельная сущность в языке.
    - этот факт используется в некоторых идиомах программирования, например **идиома safe bool**
- Указатель на статический метод - это обычный указатель на функцию, указатель на статическое поле - это обычный указатель. Все что выше - это про **нестатические** методы и поля классов
- В шаблонном коде указатель на функцию ведет себя так же (синтаксически неотличим), как объект класса с перегруженным оператором `()`. Это позволяет использовать указатели на фукнции в качестве функторов. 
    - Т.е. например, если есть шаблонный класс который ожидает объект класса с перегруженным оператором `()`, то можно передать туда указатель на функцию, если ее сингатура соответствует
- Всегда используйте `typedef/using` для определения указателей на функции/методы/поля, избегайте низкоуровневого синтаксиса таких указателей.

### <a id='toc5_6_1_'></a>[Идеома safe bool](#toc0_)

Например, мы хотим проверять валидность объектов примерно таким образом:

    if (объект)
        { /* проверка пройдена успешно, используем объект */ }
    else
        { /* проверка не пройдена, сообщаем об ошибке, завершаем программу */ }

Для этого в классе перегружаем оператор приведения к логическому типу `operator bool()` или оператор приведения к типу void* — `operator void*()` (дает нулевой указатель, если проверка провалилась). Такой подход содержит уязвимость: если есть два разных класса в которых перегружен соотвествующий оператор, то станет возможным некорретное их сравнение:

    Testable a;
    AnotherTestable b;
    if (a == b) { /* blah blah blah*/ }
    if (a < 0) { /* blah blah blah*/ }

Решение состоит в использовании `explicit` при перегрузке операторов. Тогда эта функция сработает только при явном вызове преобразования, например: `static_cast<bool>(объект)`. Кроме того, создаются шаблонные операторы сравнения `!=` и `==`. А с помощью указателя на метод вызывается обработчик ошибочного использования экземпляра в [сравнении](https://en.wikibooks.org/wiki/More_C%2B%2B_Idioms/Safe_bool#Known_Uses).

    class Testable {
        bool ok_;
        typedef void (Testable::*bool_type)() const;
        void this_type_does_not_support_comparisons() const {}  // сообщение об ошибке
    public:
        explicit Testable(bool b=true):ok_(b) {}                // конструктор с параметром и запретом неявного приведения типа 

        operator bool_type() const {
        return ok_ ? 
            &Testable::this_type_does_not_support_comparisons : 0;
        }
    };

    // не позволят использовать экземпляры в операциях сравнения
    template <typename T>
    bool operator!=(const Testable& lhs, const T&) {
        lhs.this_type_does_not_support_comparisons();	
        return false;
    }
    template <typename T>
    bool operator==(const Testable& lhs, const T&) {
        lhs.this_type_does_not_support_comparisons();
        return false;
    }

    class AnotherTestable ... // Identical to Testable.

    int main (void)
    {
        Testable t1;
        AnotherTestable t2;
        if (t1) {} // Works as expected
        if (t2 == t1) {} // Fails to compile
        if (t1 < 0) {} // Fails to compile
    }

В С++11 данная идеома максимально внедрена в синтаксис. Теперь компилятор знает, что `explicit` перегрузка `operator bool()` явно запрещает ипользовать его в операциях сравнения:

    struct Testable
    {
        explicit operator bool() const {
            return false;
        }
    };

    int main()
    {
    Testable a, b;
    if (a)      { /*do something*/ }  // this is correct
    if (a == b) { /*do something*/ }  // compiler error
    }

Данная идеома положена в основу 
```
boost::scoped_ptr
boost::shared_ptr
boost::optional
boost::tribool
```

**Задача**

Напишите возвращающую bool шаблонную функцию compare, которая принимает две константные ссылки на объекты одного типа и указатель на константный метод этого типа без параметров, который в свою очередь возвращает значение какого-то второго типа. Функция должна сравнивать объекты по значениям, которые для них вернёт соответствующий метод, и возвращать true, если значение для первого объекта оказалось меньше, чем для второго.

Пример использования функции compare:

    std::string s1("Elf");
    std::string s2("Archer");

    // сравнение строк по длине
    bool r1 = compare(s1, s2, &std::string::size); // true
    bool r2 = compare(s1, s1, &std::string::size); // false

Мой вариант:

    template <typename T1, typename T2>
    bool compare(T1 const & s1, T1 const & s2, T2 (T1::*mtd) () const) {
        return ((s1.*mtd)() < (s2.*mtd)());
    }

Хорошие варианты:

- через `typedef` определить тип метода никак не получится, но можно `using`

```
    template<typename T1, typename T2>
    using MtdPtr = T2 (T1::*)() const;

    template<typename T1, typename T2>
    bool compare(const T1 &a, const T1 &b, MtdPtr<T1,T2> mptr)
    {
        return (a.*mptr)() < (b.*mptr)();
    }
```
- через constexpr (C++11) - если значения параметров возможно посчитать на этапе компиляции, то возвращаемое значение также должно посчитаться на этапе компиляции, иначе - считается в рантайме

```
    template <typename T1, typename T2>
    constexpr bool compare(const T1& s1, const T1& s2, T2 const mtd)
    {
        return (s1.*mtd)() < (s2.*mtd)();
    }
```
**Напоминалка**

Правило парсинга `const` - **применяется к тому, что слева, а если слева ничего нет, то к тому что справа**, поэтому:

    const Object* obj;          // can't change data - без пробела даже выглядит по уродски
    Object* const obj;          // can't change pointer
    const Object* const obj;    // can't change data or pointer
    
    Object const *obj;          // == const Object* obj

- поэтому мне больше нравится ставить `const` справа, потому что он так изначально задумывался. Это квалификатор, он должен менять что-то что уже есть, а читаем мы слева направо!

## <a id='toc5_7_'></a>[Пространства имён](#toc0_)

### <a id='toc5_7_1_'></a>[Пространства имён](#toc0_)

Например, у нас есть 2 компоненты, отвечающие за звук и графику. Функции там могут быть близкие по смыслу и одинаково называться (send/proceed и т.п.) и иметь одинаковую сигнатуру. Мы хотим их разграничить и не выдумывать длинные названия. Для этого помещаем этим функции в разные пространства имен. Если этого не сделать, то но этапе линковки получим ошибку двойного определения функции. 

Реже могут быть и более сложные коллизии, которые не вылезут на этапе линковки - когда два класса в разных компонентах имеют разную реализацию, например, в них определен разный инлайн деструктор. При компиляции из них останется только один (из одинаковых инлайн-функции выберется любая) и использоваться будет только он, а это UB. 

**Напоминалка**
- Классы и inline функции могут определяться в более чем одной единице трансляции, но определения обязаны совпадать. Ошибки такого рода никто не выявляет, они просто ведут к неопределенному поведению. 
- Методы, код которых определен внутри класса, считаются/являются inline-методами. Но можно определить метод отдельно с использованием директивы inline.
- Объявление - позволяет компилятору обращаться к элементу по имени (на этапе создания объектных файлов), на этапе линковки вместо имен подставляются адреса и они должны быть к этому моменту известны.
    - чтобы обращаться к элементу по указателю достаточно только имени из объявления (размер указателя зависит только от архитектуры 32/64 бит)
    - чтобы получить значение по указателю уже нужно знать размер объекта, а значит он должен быть не только объявлен, но и определн
    - предварительное объявление позволяет при изменении определения не перекомпилировать те единицы трансляции, где требуется только объявление
    - видимость объявления между единицами трансляции устанавливается модификатором `extern`

**Пространства имён** (namespaces) — это способ разграничения областей видимости имён в C++. С использование пространств имен об уникальности идентификаторов нужно заботиться только в рамках пространства имен.

Имена в C++:
1. имена переменных и констант,
2. имена функций,
3. имена структур и классов,
4. имена шаблонов,
5. синонимы типов (typedef-ы),
6. enum-ы и union-ы,
7. имена пространств имён (т.е. их можно вкладывать друг в друга).

В C++11 еще есть "псевдонимы" ( `alias` ) для типов, шаблонов, и пространств имен. Причем template alias похоже создает новый шаблон.  В C++98 есть имена меток, хотя они локальные для функций.

## <a id='toc5_8_'></a>[Примеры](#toc0_)

В C нет понятия пространства имен, для избежания конфликта имён используются префиксы. К примеру, имена в C библиотеке Expat начинаются с XML_.

    struct XML_Parser;
    int XML_GetCurrentLineNumber(XML_Parser * parser);

В C++ это можно было бы записать так:

    namespace XML {
        struct Parser;
        int GetCurrentLineNumber(Parser * parser);
        ... // тут можно работать с объявленными элементами без префикса
    }

Тогда полные имена структуры и функции снаружи пространства имен будут соответственно:

    XML::Parser
    XML::GetCurrentLineNumber

**Любопытный факт:** в Qt все имена классов начинаются с префикса Q (QWidget, QString, QVector ...)  Как говорят авторы, причина этому в том, что в ранних версиях C++ не было пространств имен.

## <a id='toc5_9_'></a>[Описание пространства имён](#toc0_)

1. Пространства имён могут быть вложенными:
```
    namespace items { namespace food {
        struct Fruit {...};
    }}
    items::food::Fruit apple("Apple");
```
2. Определение пространств имён можно разделять (описывать по частям):
```
    namespace weapons { struct Bow { ... }; }
    namespace items {
        struct Scroll { ... };
        struct Artefact { ... };
    }
    namespace weapons { struct Sword { ... }; }
```

- со стандарта С++17 упростили синтаксис вложенения пространств имен, т.е. можно будет так:
```
    namespace items::food {
      struct Fruct { /*...*/ };
    }
```
3. Классы и структуры определяют одноимённый namespace.
- правда к нему нельзя применять `using namespace` для доступа к методам, только доступ через `::`

## <a id='toc5_10_'></a>[Доступ к именам](#toc0_)

Доступ к именам
1. Внутри того же `namespace` все имена доступны напрямую.
2. `NS::` позволяет обратиться внутрь пространства имён `NS`.
```
    namespace NS { 
        int foo() { return 0; };
        int bar() { return foo(); };
    }
    int i = NS::foo();
```
3. Оператор `::` позволяет обратиться к глобальному пространству имён.
```
    struct Dictionary {...};            // глобальный словарь
    
    namespace items
    {
        struct Dictionary {...};        // локальный словарь
        
        ::Dictionary globalDictionary;  // обращение к глобальному словарю
    }
```

\*) оператора для доступа к объемлющему неймспейсу в С++ нет, только по имени...

## <a id='toc5_11_'></a>[Поиск имён](#toc0_)

**Поиск имён** — это процесс разрешения имени.
1. Если такое имя есть в текущем namespace
- выдать **все** одноимённые сущности в текущем namespace
- завершить поиск
2. Если текущий namespace — глобальный
- завершить поиск c ошибкой компиляции
3. Текущий namespace ← родительский namespace.
4. Перейти на шаг 1.

**Пример:**
- есть 3 определения `foo()` с разной сигнатурой
```
int foo(int i) { return 1; }

namespace ru
{
    int foo(float f) { return 2; }

    int foo(double a, double b) { return 3; }

    namespace spb {
        int global = foo(5);    // 2
    }
}
```

1. `spb::foo` отсутствует
2. в `ru::foo` компилятор найдет два определения `foo` и запустит процесс перегрузки функции (подбор по сигнатуре)
3. до максимально подходящего по сигнатуре определения `foo` в глобальном неймспейсе поиск имен не дойдет
- если бы вместо `foo(float f)` была бы несовместимая сигнатура, то была бы ошибка компиляции, в глобальный неймспейс компилятор не полез бы (`error: no matching function for call to ‘foo(int)’`)
- если бы в `spb` вообще не было бы `foo()` то подходящая нашлась бы в глобальном пространстве имен

**Важно**: поиск продолжается до первого совпадения.

В перегрузке участвуют только найденные к этому моменту функции.



## <a id='toc5_12_'></a>[Ключевое слово using](#toc0_)

Позволяет добавить все имена/одно имя из другого пространства имен в текущее простанство имен.


Существуют два различных использования слова `using`.
```
namespace ru
{
    namespace msk {
        int foo(int i) { return 1; }
        int bar(int i) { return -1; }
    }

    using namespace msk;    // все имена из msk
    using msk::foo;         // только msk::foo

    int foo(float f) { return 2; }
    int foo(double a, double b) { return 3; }

    namespace spb {
        int global = foo(5);    // 1: перегрузка функций выберет ставший доступным msk::foo(int i)
    }
}
```
\*) без подключения имена ищутся только в родительском простанстве имен, даже если в нем есть другой дочерний с подхощями именами без подключения их не будет видно

**Замечание**

- Если функция объявлена в подключенном пространстве имен и подключена по имени, то в текущем ее объявлять нельзя. Это запрет стандарта и он следует из:
    - `using namespace msk` - все имена доступны как если бы они были объявлены в родительском простанстве имен (т.е. в нем ищется только после поиска в текущем ПИ)
        - **можно** переопределить в текущем ПИ
    - `using msk::foo` - имя доступно для поиска как если бы оно было объявлено в текущем пространстве имен (т.е. участвует в самом первом поиске в рамках текущего ПИ)
        - **нельзя** переопределить в текущем ПИ

## <a id='toc5_13_'></a>[Поиск Кёнига](#toc0_)

Выше описание поиска в ПИ в общих словах. **Поиск Кёнига** - это строгая модель как осуществляет поиск в ПИ

Пусть есть ПИ для работы с графикой:

    namespace cg {
        struct Vector2 {...};
        Vector2 operator+(Vector2 a, Vector2 const& b);
    }

Использование графических примитивов снаружи ПИ:

    cg::Vector2 a(1,2);
    cg::Vector2 b(3,4);
    b = a + b;                  // эквивалентно: b = operator+(a, b)
    b = cg::operator+(a, b);    // OK

Как без использования `using` или указания ПИ (как во втором вызове) компилятор найдет данный конкретный `operator+` из `cg` в первом вызове, ведь он не должен по идее заходить в дочернее ПИ, а искать в текущем и подниматься в родительские? А компилятор найдет, как? Кроме того, для оператора `+` и нельзя указать ПИ напрямую...

Для этого в С++ используется **Argument-dependent name lookup (ADL, Поиск Кёнига)**

При поиске имени функции на первой фазе рассматриваются имена из текущего пространства имён и пространств имён, к **которым принадлежат аргументы функции**.

- используется только для поиска имен фукнций (не переменных)
- т.е. это такое исключение из общего правила поиска в ПИ, когда для функций автоматически подключается ПИ, откуда взяты все аргументы этих функций
- именно этот фокус позволяет использовать конструкции вида `std::cout << ... << std::endl;`, где для оператора `<<` реализация берется из `std::` безо всяких `using`


## <a id='toc5_14_'></a>[Безымянный namespace](#toc0_)

Пространство имён с гарантированно уникальным именем, которое генерирует компилятор. Оно автоматически подключается к объемлющему ПИ.

    namespace { // безымянный namespace
        struct Test { std::string name; };
    }

Это эквивалентно:

    namespace $GeneratedName$ {
        struct Test { std::string name; };
    }
    using namespace $GeneratedName$;

**Зачем это нужно**

Безымянные пространства имён — замена для `static`.
- статические функции и глобальные переменные - это такие, которые доступны только в рамках одной единицы трансляции и нужны для предотвращения конфликтов имен между единицами трансляции
- но такой фокус не пройдет с классами, т.к. `static` работает с помощью внутренней линковки, а классы сами по себе не линкуются, это такие абстракции над данными и методами. Как же локализовать класс целиком в рамках одной единицы трансляции, чтобы в другой можно было использовать класс с таким же именем, но с другой реализацией?
- мы кладем такие классы в безымянное ПИ и компилятор уже не будет видеть их из других единиц трансляции
- тоже самое можно сделать и с функциями и глобальными переменными вместо использования `static`, просто для классов этот механизм беальтернативный в отличие от
- к статическим методами этот подход не относится, только к классам целиком, если нужны статические методы - то только `static`

**Напоминалка**
- внешняя/внутренняя линковка - это видимость символов для компоновщика при обработке файлов
- внешняя линковка (ключевое слово `extern`) - это когда символ виден всем единицам трансляции (и проверяется на конфликты глобально) 
    - такой символ должен быть определен 1 раз в одной единице трансляции, обычно в файле реализации
    - неконстантные глобальные для данной единицы трансляции переменные по умолчанию имеют внешнюю линковку (`int x;` == `extern int x{};`, `{}` отражают тонкость, что `x` тут и объявлено и определено, просто объявление вылгядело бы как `extern int x;`) 
    - глобальные в рамках ед.транс. константы имеют внутреннюю линковку по умолчанию
- внутренняя линковка (ключевое слово `static`) - обычно исп. в заголовочных файлах
    - гарантирует, что все, кто подлючит этот заголовок получат свою копию символа

## <a id='toc5_15_'></a>[Заключение](#toc0_)

1. Используйте пространства имён для исключения конфликта имён.
2. Помните, что поиск имён прекращается после первого совпадения. Используйте using и полные имена.
3. Не используйте `using namespace` в заголовочных файлах, только в файлах с кодом.
4. Всегда определяйте операторы в том же пространстве имён, что и типы, для которых они определены.
5. Используйте безымянные пространства имён для маленьких локальных классов и как замену слова `static`.
6. Для длинных имён `namespace`-ов используйте синонимы: `namespace csccpp17 = ru::spb::csc::cpp17;` (алиасы для длинных путей)

**Задача**

Предположим, что наша программа состоит из файлов `header.hpp, file1.cpp, file2.cpp и file3.cpp`. При этом файл `header.hpp` включается во все `.cpp` файлы. Внутри `header.hpp` мы написали следующий код.

    namespace 
    {
        struct Test 
        { 
            std::string data; 
        };

        inline void foo() { ... }

        static void bar() { ... }

        void foobar() { ... }
    }

**Замечание**: давайте предполагать, что inline функция не будет встроена компилятором.

Тогда мы можем утверждать:
- В каждой единице трансляции будет свой собственный класс Test.
- После компиляции в программе будет три различные функции foo.
- Функцию foobar можно определить в заголовочном файле, т.к. она определена в безымянном пространстве имён.
- В результате компиляции в каждой единице трансляции будут свои функции foo, bar и foobar.
```
    $ ./file3 
    file1.cpp
    - foo   : 0x401162
    - bar   : 0x401169
    - foobar: 0x401170
    file2.cpp
    - foo   : 0x40125b
    - bar   : 0x401262
    - foobar: 0x401269
    file3.cpp
    - foo   : 0x401354
    - bar   : 0x40135b
    - foobar: 0x401362
```